In [ ]:
!pip install -q efficientnet



In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
from kaggle_datasets import KaggleDatasets

import efficientnet.tfkeras as efn

pd.set_option("display.max_columns", None)

# Dataset Exploration

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')


In [ ]:
train.head()


In [ ]:
print(len(train))
print(train.columns)
print(train['labels'].value_counts().plot.bar())

các loại nhãn khác nhau


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
label_split = train.labels.apply(lambda x: x.split()) #chia 1 chuỗi các nhãn thành nhiều nhãn nếu có dấu cách
label_split.head()


In [ ]:
trans_label = MultiLabelBinarizer().fit(label_split)
labels = pd.DataFrame(trans_label.transform(label_split), columns=trans_label.classes_)

labels.sum().plot.bar(title='Target Class Distribution');

In [ ]:
labels.sum(axis=1).value_counts().plot.bar(title='Distribution of Number of Labels per Image');

In [ ]:
# fig1 = plt.figure(figsize=(26,10))

# for i in range(1, 13):
    
#     rand =  random.randrange(1, 18000)
#     sample = os.path.join('../input/plant-pathology-2021-fgvc8/train_images', train['image'][rand])
    
#     img = PIL.Image.open(sample)
    
#     ax = fig1.add_subplot(4,3,i)
#     ax.imshow(img)
    
#     title = f"{train['labels'][rand]}{img.size}"
#     plt.title(title)
    
#     fig1.tight_layout()

size các ảnh khác nhau


# Preprocessing and Augmentation

In [ ]:

train_df = pd.concat([train['image'], labels], axis=1)
train_df.head()

In [ ]:
w_target = 256
h_target = 256
batch_size = 32

In [ ]:
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    validation_split=0.03,
    rescale=1./255)

train_generator = image_data_generator.flow_from_dataframe(
    dataframe=train_df,
    directory='../input/resized-plant2021/img_sz_512',
    x_col='image',
    y_col=train_df.columns.tolist()[1:],
    class_mode='raw',
    color_mode="rgb",
    target_size=(h_target, w_target),
    batch_size=batch_size,
    subset='training'
)

valid_generator = image_data_generator.flow_from_dataframe(
    dataframe=train_df,
    directory='../input/resized-plant2021/img_sz_512',
    x_col='image',
    y_col=train_df.columns.tolist()[1:],
    class_mode='raw',
    color_mode="rgb",
    target_size=(h_target, w_target),
    batch_size=batch_size,
    subset='validation'
)

In [ ]:
!pip install -q efficientnet


# Modelling

In [ ]:
# inputs = tf.keras.Input(shape=(h_target, w_target, 3)
Effnet=efn.EfficientNetB4(
        include_top=False,
        input_shape=(h_target,w_target, 3),
        weights='noisy-student',
        pooling='avg')
# Effnet= tf.keras.applications.EfficientNetB4(weights='imagenet',include_top=False, input_shape=(h_target,w_target, 3))
x = Effnet.output
# for layer in VGG_16.layers:
#     layer.trainable=False
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.8)(x)
outputs = tf.keras.layers.Dense(6, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=Effnet.input, outputs=outputs)

model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=6, average='micro', threshold=0.5)

rlp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min', patience=2, verbose=1, factor=0.01)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=5, verbose=1, restore_best_weights=True)

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001), 
              metrics= [f1])

history = model.fit(train_generator,validation_data=valid_generator, epochs=10, callbacks=[rlp,es])


In [ ]:
fix, ax = plt.subplots(figsize=(20, 6))
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(ax=ax, title='Model Loss Curve')

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0014), 
              metrics= [f1])
history = model.fit(train_generator,validation_data=valid_generator, epochs=25, callbacks=[rlp,es])


In [ ]:
fix, ax = plt.subplots(figsize=(20, 6))
pd.DataFrame(history.history)[['loss','val_loss']].plot(ax=ax, title='Model Loss Curve')

# Submission

In [ ]:
submissions = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submissions.head()

In [ ]:
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_dataframe(
    submissions,
    directory = '../input/plant-pathology-2021-fgvc8/test_images',
    x_col="image",
    y_col=None,
    target_size=(h_target, w_target),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    shuffle=False,
    batch_size=batch_size
)

preds = model.predict(test_generator)
print(preds)

In [ ]:
thresh = 0.5
for i in range(3):
    submissions.iloc[i, 1] = ' '.join(train_df.columns[1:][preds[i] >= thresh])
    if submissions['labels'][i] == '':
        submissions['labels'][i] = ' '.join(train_df.columns[1:][preds[i] >= np.max(preds[i])])
        
submissions.to_csv('submission.csv', index=False)    


In [ ]:
submissions

In [ ]:
model.save('EffnetB4.h5')